In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
! pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00


In [ ]:
! pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 7.9 MB/s eta 0:00:00


In [ ]:
! pip install datasets transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.5 MB/s eta 0:00:00


In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
import torch
import evaluate
from datasets import Dataset, Features, Value, ClassLabel

In [ ]:
def load_data(file_path):
    df = pd.read_csv(file_path)
    df.rename(columns=lambda x: x.strip(), inplace=True)
    df = df[['text', 'labels']]
    return df

In [ ]:
final_training_set = load_data('final_training_set_fully_balanced_shuffled.csv')

In [ ]:
final_validation_set = load_data('final_validation_set.csv')

In [ ]:
class_names = ['Non-anti-LGBT+ content', 'Homophobia', 'Transphobia']

In [ ]:
final_training_set = Dataset.from_pandas(final_training_set, features=Features({'text': Value('string'), 'labels': ClassLabel(num_classes=3, names=class_names)}))
final_validation_set = Dataset.from_pandas(final_validation_set, features=Features({'text': Value('string'), 'labels': ClassLabel(num_classes=3, names=class_names)}))

In [ ]:
print(final_training_set)

Dataset({
    features: ['text', 'labels'],
    num_rows: 120501
})


In [ ]:
print(final_validation_set)

Dataset({
    features: ['text', 'labels'],
    num_rows: 9964
})


In [ ]:
checkpoint = "distilbert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_datasets_train = final_training_set.map(tokenize_function, batched=True)
tokenized_datasets_validation = final_validation_set.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer)

Map:   0%|          | 0/120501 [00:00<?, ? examples/s]

Map:   0%|          | 0/9964 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    "test-trainer",
    save_total_limit=20,
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    metric_for_best_model="eval_loss",
    num_train_epochs=15,
    learning_rate=2e-5,
    weight_decay=0.01,
)

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_validation,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.154700,0.359044,0.885990
2,0.082000,0.478257,0.890707
3,0.049400,0.497375,0.905861
4,0.033800,0.605533,0.905961
5,0.026800,0.700022,0.903854
6,0.020500,0.638885,0.909675
7,0.007400,0.811135,0.906062
8,0.007300,0.745080,0.909574
9,0.011200,0.823636,0.901345
10,0.007700,0.824136,0.911782


KeyboardInterrupt: 

In [ ]:
trainer.save_model("final_model")